In [ ]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

In [ ]:
df = pd.read_csv('/Users/avacheevers/Desktop/clearcut_categories/SQL_input/Collagen.csv')
df['Month'] = pd.to_datetime(df['Month'])
category = 'Collagen'

In [ ]:
# finds year of most recent data
year = str(df.sort_values('Month', ascending = 0).Month[0])[:4]
last_year = str(int(year)-1)

In [ ]:
# finds month of most recent data
month = str(df.sort_values('Month', ascending = 0).Month[0])[5:7]

In [ ]:
if month in ['12', '01', '02']:
    to_calc = ['Q3Q4', 'Q2Q3', 'Q1Q2', 'Q4Q1']
elif month in ['03','04' ,'05']:
    to_calc = ['Q4Q1', 'Q3Q4', 'Q2Q3', 'Q1Q2']
elif month in ['06','07', '08']:
    to_calc = ['Q1Q2', 'Q4Q1', 'Q3Q4', 'Q2Q3']
elif month in ['09','10', '11']:
    to_calc = ['Q2Q3', 'Q1Q2', 'Q4Q1', 'Q3Q4']

In [ ]:
def find_mask(quarter):
    if quarter == 'Q1':
        mask = (df['Month'] >= year+'-01-01') & (df['Month'] < year+'-04-01')
    if quarter == 'Q2':
        mask = (df['Month'] >= year+'-04-01') & (df['Month'] < year+'-07-01')
    if quarter == 'Q3':
        mask = (df['Month'] >= last_year+'-07-01') & (df['Month'] < last_year+'-09-01')
    if quarter == 'Q4':
        mask = (df['Month'] >= last_year+'-09-01') & (df['Month'] < year+'-01-01')
    return mask

def rev_format(rev):
    if rev > 10000000:
        formatted = str(round(rev/1000000,1))+' MM'
    elif rev > 500000: 
        formatted = str(round(rev/1000000,2))+' MM'
    else:
        formatted = str(round(rev/1000,1))+' K'
        
    return formatted

In [ ]:
def calculate_fields(quarters):
    a_quarter_name = quarters[:2]
    b_quarter_name = quarters[2:]
    
    a_mask = find_mask(a_quarter_name)
    a = df.loc[a_mask]
    
    b_mask = find_mask(b_quarter_name)
    b = df.loc[b_mask]
    
    a_agg = a.groupby(['tagbrand']).agg({'Revenue': ['sum']})
    a_agg.columns = ["_".join(x) for x in a_agg.columns.ravel()]
    a_agg = a_agg.reset_index()
    
    b_agg = b.groupby(['tagbrand']).agg({'Revenue': ['sum']})
    b_agg.columns = ["_".join(x) for x in b_agg.columns.ravel()]
    b_agg = b_agg.reset_index()
    
    ba = b_agg.merge(a_agg, how = 'left', on = ['tagbrand'])
    ba.columns = ['tagbrand', 'rev_b', 'rev_a']
    
    ba.rev_b = round(ba.rev_b, 0)
    ba.rev_a = round(ba.rev_a, 0)
    
    ba['market_share_b_perc'] = round(100*(ba.rev_b / ba.rev_b.sum()),2)
    ba['market_share_a_perc'] = round(100*(ba.rev_a / ba.rev_a.sum()),2)
    ba['category_growth_perc'] = round(100*((ba.rev_b.sum() - ba.rev_a.sum()) /ba.rev_a.sum()),2)
    ba['brand_growth_perc'] = round(100*((ba.rev_b - ba.rev_a)/ba.rev_a),2)
    ba['growth_gap'] = ba.brand_growth_perc - ba.category_growth_perc
    ba['market_share_perc_change'] = round(100*((ba.market_share_b_perc-ba.market_share_a_perc) / ba.market_share_a_perc),2)

    ba = ba.sort_values('market_share_a_perc', ascending = 0)
    ba = ba.reset_index()
    ba['rank_a'] = (ba.index)+1
    
    ba = ba.sort_values('market_share_b_perc', ascending = 0)
    ba = ba.reset_index()
    ba['rank_b'] = (ba.index)+1
    
    ba['rev_b_formatted'] = ba.rev_b.apply(lambda x: rev_format(x))
    ba['rev_a_formatted'] = ba.rev_a.apply(lambda x: rev_format(x))
    
    ba['market_share_below_val'] = ba.market_share_b_perc.shift(-1)
    ba['market_share_below_tagbrand'] = ba.tagbrand.shift(-1)
    ba['brand_growth_below_val'] = ba.brand_growth_perc.shift(-1)
    
    ba['market_share_two_below_val'] = ba.market_share_b_perc.shift(-2)
    ba['market_share_two_below_tagbrand'] = ba.tagbrand.shift(-2)
    
    ba['rev_above_val'] = ba.rev_b_formatted.shift(1).astype(str)
    ba['rev_two_above_tagbrand'] = ba.tagbrand.shift(2).astype(str)
    ba['rev_three_above_tagbrand'] = ba.tagbrand.shift(3).astype(str)
    ba['rev_four_above_tagbrand'] = ba.tagbrand.shift(4).astype(str)
    ba['rev_above_tagbrand'] = ba.tagbrand.shift(1)
    ba['market_share_above_val'] = ba.market_share_b_perc.shift(1)

    
    ba['hook_1'] = 'The '+ba.tagbrand+' brand generated $'+ba.rev_b_formatted+' last quarter with '+ba.brand_growth_perc.apply(lambda x: str(x))+'% in-category growth. If the following information was available for all your competitors, would that be helpful or did I miss the mark? '
    
    ba['hook_2'] = 'In '+b_quarter_name+ ' our analysis shows that '+ba.tagbrand+' market share increased to '+ba.market_share_b_perc.apply(lambda x: str(x))+'% with '+ba.market_share_below_tagbrand+' sitting at '+ba.market_share_below_val.apply(lambda x: str(x))+'%. What if we could help you gain back market-share from a competitor who is encroaching on your brand and help make sure '+ba.market_share_below_tagbrand+' does not catch up?'
    
    ba['hook_3'] =  'In '+b_quarter_name+ ' our analysis shows that '+ba.tagbrand+' market share increased to '+ba.market_share_b_perc.apply(lambda x: str(x))+'%. The market share of '+ba.market_share_below_tagbrand+' is currently at '+ba.market_share_below_val.apply(lambda x: str(x))+'% with '+ba.market_share_two_below_tagbrand+' trailing behind at '+ba.market_share_two_below_val.apply(lambda x: str(x))+'%. What if we could help you gain back market-share from a competitor who is encroaching on your brand and help make sure '+ba.market_share_below_tagbrand+' does not catch up?' 

    ba['hook_4'] = 'Last quarter '+ba.rev_above_tagbrand+' generated $'+ ba.rev_above_val.apply(lambda x: str(x))+' in revenue. The '+ba.rev_above_tagbrand+' now has '+(ba.market_share_above_val - ba.market_share_b_perc).apply(lambda x: str(round(x,2)))+' percentage points more market share than '+ba.tagbrand+' . What if you could strategically outmaneuver your competition to gain market share using e-commerce business intelligence your competitors don’t have?'
    
    ba['hook_5'] = 'You are currently ranked #'+ba.rank_b.apply(lambda x: str(x))+' in '+category+'  by monthly revenue. With our ecomm-focused data and insights, we can help your brand gain the #'+ba.rank_b.apply(lambda x: str(x-1))+' spot from '+ba.rev_above_tagbrand+' by increasing your market share.'
    
    ba['hook_6'] = 'Are you aware that '+ba.market_share_below_tagbrand.apply(lambda x: str(x))+' captured '+ba.brand_growth_below_val.apply(lambda x: str(x))+'%  category growth in the last quarter narrowing '+ba.tagbrand+'\'s gap in growth down to '+(ba.brand_growth_perc - ba.brand_growth_below_val).apply(lambda x: str(round(x,2)))+ ' percentage points? Clearcut Analytics makes it easy to garner trends and insights that align with their customers’ actual buying patterns. We help unlock eCommerce market intelligence to track your competitors’ every move and discover emerging category trends.'
    
    ba['hook_7'] = 'Congrats on grossing '+ba.rev_b_formatted+' and increasing quarterly growth in the '+category+' category to '+ba.brand_growth_perc.apply(lambda x: str(round(x,2)))+'%. However compared to the category overall growth your brand is still behind by '+(ba.category_growth_perc - ba.brand_growth_perc).apply(lambda x: str(round(x,2)))+' percentage points.  How are you planning to take advantage of the recent eCommerce acceleration within VMS while your brick-and-mortar business likely plays catch-up?'
    
    ba['hook_8'] = 'I noticed '+ba.tagbrand+'  was ranked #'+ba.rank_b.apply(lambda x: str(x))+' in '+category+' last quarter.  If I could provide you with a portal that showcases '+ba.rev_above_tagbrand+', '+ba.rev_two_above_tagbrand+', '+ba.rev_three_above_tagbrand+', and '+ba.rev_four_above_tagbrand+'\'s e-commerce sales & market share per category, does a quick email exchange make sense to learn about how Clearcut Analytics is helping VMS brands grow with the new online space?'
    
    ba = ba[['tagbrand',
             'hook_1',
             'hook_2',
             'hook_3',
             'hook_4',
             'hook_5',
             'hook_6',
             'hook_7',
             'hook_8',
            'rev_b',
            'rev_a',
            'rev_b_formatted',
            'rev_a_formatted',
            'market_share_b_perc',
            'market_share_a_perc',
            'category_growth_perc',
            'brand_growth_perc',
            'growth_gap',
            'market_share_perc_change',
            'rank_b',
            'rank_a']]
    ba.columns = ['Brand',
                 'Hook 1',
                 'Hook 2',
                 'Hook 3',
                 'Hook 4',
                 'Hook 5',
                 'Hook 6',
                 'Hook 7',
                 'Hook 8',
                 'Revenue '+b_quarter_name,
                 'Revenue '+a_quarter_name,
                 'Revenue '+b_quarter_name+' Formatted',
                 'Revenue '+a_quarter_name+' Formatted',
                 'Market Share (%)'+b_quarter_name,
                 'Market Share (%)'+a_quarter_name,
                 'Category Growth (%)'+a_quarter_name+b_quarter_name,
                 'Brand Growth (%)'+a_quarter_name+b_quarter_name,
                 'Brand-Category Growth Gap',
                 'Market Share % Change',
                 'Rank in'+b_quarter_name,
                 'Rank in'+a_quarter_name
                 ]
    
    return ba

In [ ]:
df1 = calculate_fields('Q1Q2')

In [ ]:
df1.to_csv('/Users/avacheevers/Desktop/clearcut_categories/'+category+'_hooks.csv')

#### 

In [ ]:
df1